<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Kleines-Beispiel" data-toc-modified-id="Kleines-Beispiel-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Kleines Beispiel</a></span></li></ul></div>

In [1]:
import sys
import os

try:
    os.chdir(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    sys.path.append(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    
import warnings
warnings.filterwarnings("ignore")

In [54]:
import nglview as nv
import MDAnalysis as mda
import numpy as np
import time
import ipywidgets as widgets
from scipy.sparse import csr_array
from numpy.typing import NDArray
import matplotlib

from molgri.molecules.transitions import SimulationHistogram, MSM, SQRA
from molgri.plotting.molecule_plots import TrajectoryPlot
import pandas as pd
import matplotlib.pyplot as plt
from molgri.molecules.parsers import FileParser, ParsedEnergy, XVGParser

from molgri.paths import PATH_OUTPUT_PT, OUTPUT_PLOTTING_DATA, PATH_OUTPUT_LOGGING
from molgri.space.fullgrid import FullGrid
from molgri.space.utils import k_argmin_in_array, k_argmax_in_array

Erstmals: wir brauchen eine HF Monomer Datai.

In [4]:
%pycat input/HF.xyz

Nun lass zwei PTs (klein, groß) produzieren:
- klein: -o 42 -t "linspace(0.1, 0.4, 10)" -b 8
- groß: -o 162 -t "linspace(0.1, 0.4, 20)" -b 40

## Kleines Beispiel
- obtain full grid
- show full grid coordinates per frame in the corner

In [8]:
small_name = "HF_HF_0010"
b_grid = "8"
o_grid = "12"
t_grid = "linspace(0.2, 0.4, 3)"

In [4]:
%run molgri/scripts/generate_pt.py -m1 HF -m2 HF --recalculate -o $o_grid -b $b_grid -t "linspace(0.2, 0.4, 3)"

Saved the log file to output/data/logging/HF_HF_0010.log
Saved pseudo-trajectory to output/data/pt_files/HF_HF_0010.xtc and structure file to output/data/pt_files/HF_HF_0010.gro
Timing the generation of Pt HF_HF_0010: 0:00:00.479715 hours:minutes:seconds


In [6]:
# großeres beispiel
big_name = "HF_HF_0011"
b_grid_b = "272"
o_grid_b = "162"
t_grid_b = "linspace(0.1, 0.4, 20)"

In [4]:
%run molgri/scripts/generate_pt.py -m1 HF -m2 HF --recalculate -o $o_grid_b -b $b_grid_b -t "linspace(0.1, 0.4, 20)"

Saved the log file to output/data/logging/HF_HF_0011.log
Saved pseudo-trajectory to output/data/pt_files/HF_HF_0011.xtc and structure file to output/data/pt_files/HF_HF_0011.gro
Timing the generation of Pt HF_HF_0011: 0:24:59.522670 hours:minutes:seconds


In [3]:
%run molgri/scripts/generate_pt.py -m1 H2O -m2 H2O --recalculate -o "162" -b "272" -t "linspace(0.1, 0.4, 20)"

Saved the log file to output/data/logging/H2O_H2O_0092.log
Saved pseudo-trajectory to output/data/pt_files/H2O_H2O_0092.xtc and structure file to output/data/pt_files/H2O_H2O_0092.gro
Timing the generation of Pt H2O_H2O_0092: 0:25:25.037286 hours:minutes:seconds


In [ ]:
#WATER

In [27]:
from MDAnalysis import transformations

u = mda.Universe(r"/home/hanaz63/nobackup/gromacs/H2O_H2O_0095_10000/H2O_H2O_0095.gro", 
                 r"/home/hanaz63/nobackup/gromacs/H2O_H2O_0095_10000/output.xtc")

#reconnect molecules


ag = u.atoms
# we will use mass as weights for the center calculation
workflow = (transformations.center_in_box(ag, center='mass'),)
u.trajectory.add_transformations(*workflow)


v = nv.show_mdanalysis(u)
v.add_unitcell()
v

NGLWidget(max_frame=10000)

In [5]:
%matplotlib notebook

In [86]:
class ViewManager:
    
    """
    NGLViewer is very useful but not very convinient for displaying particular frames of a trajectory together, 
    in particular color schemes etc. This class accepts a MDA Universe and knows how to extract frames from it.
    The plotting functions then accept indices (one or several) for this trajectory and display them in a 
    particular way (overlapping, sequential ...)
    """
    
    def __init__(self, u: mda.Universe):
        self.u = u
        self.fresh_view()
    
    def fresh_view(self):
        """
        Run this when you want to start a new view and discard an old one.
        """
        self.view = nv.NGLWidget()
    
    def get_ith_frame(self, i: int) -> mda.Universe:
        """
        The most important method acting on self.u. Get the Universe object containing of all the atoms 
        but only a single frame (i-th frame).
        
        Args:
            i (int): the index of the frame wanted
        
        Returns:
            a Universe containing the i-th frame of self.u
        """
        self.u.trajectory[i]
        all_atoms = self.u.select_atoms('all')
        new_u = mda.Merge(self.u.atoms)
        return new_u
    
    def _add_coordinate_axes(self):
        """
        Helper method. Add the x, y and z axes at origin to a NGLView.
        """
        
        # arguments of add_arrow are: start position, end position, color (in RGB), radius of arrow head
        # arguments of add_label are: position, color (in RGB), size, text
        
        # X-axis is red
        self.view.shape.add_arrow([0, 0, 0], [1, 0, 0],[ 1, 0, 0 ], 0.1)
        self.view.shape.add_label([1, 0, 0], [1, 0, 0], 1.5, 'x')
        
        # Y-axis is green
        self.view.shape.add_arrow([0, 0, 0], [0, 1, 0],[0, 1, 0 ], 0.1)
        self.view.shape.add_label([0, 1, 0], [0, 1, 0], 1.5, 'y')
        
        # Z-axis is blue
        self.view.shape.add_arrow([0, 0, 0], [0, 0, 1],[0, 0, 1 ], 0.1)
        self.view.shape.add_label([0, 0, 1], [0, 0, 1], 1.5, 'z')

    def plot_ith_frame(self, frame_i: int, axes: bool = True, **kwargs):
        """
        Plot i-th frame of self.u, adding to self.view.
        
        Args:
            - i: index of the frame
            - axes: if True, draw x, y and z axes
        """
        ith_atoms = self.get_ith_frame(frame_i)
        
        self.view.add_component(ith_atoms, default_representation=False)
        # the index is there in order to only affect the last added representation
        self.view[-1].add_representation("ball+stick", **kwargs)
        if axes:
            self._add_coordinate_axes()
        return self.view
    
    def _add_optional_representation_parameters(self, my_index: int, colors: list, opacities: list):
        """
        Helper method if you want to plot several view and pass arguments to them.
        """
        kwargs = {}
        if colors is not None:
            kwargs["color"] = colors[my_index]
        if opacities is not None:
            kwargs["opacity"] = opacities[my_index]
        return kwargs
        
    
    def plot_frames_sequential(self, list_indices: list, colors: list = None, opacities: list = None):
        """
        Plot several frames of the self.u next to each other. Automatically ngo to next now if you have too
        many frames to display in one row.
        
        Args:
            - list_indices: a list of integers, each an frame index to be displayed
            - colors: a list of colors (must be same length as list_indices) or None (default)
            - opacities: a list of opacities (must be same length as list_indices) or None (default)
        """
        
        # settings that are important so that rows with too many images nicely overflow in the next row
        box = widgets.Box(layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
        box.overflow_x = 'auto'

        all_views = []
        for li, list_i in enumerate(list_indices):
            self.fresh_view()
            # add optional parameters
            kwargs = self._add_optional_representation_parameters(li, colors, opacities)
            neig_view = self.plot_ith_frame(list_i, **kwargs)
            # this is also important for nice arragement of figures
            neig_view.layout.width = "200px"
            all_views.append(neig_view)
        
        
        # sync all views (so that all plots move if you move any)
        for v in all_views:
            v._set_sync_camera(all_views)
        
        box.children=[i for i in all_views]
        display(box)
    
    def plot_frames_overlapping(self, list_indices: list, colors: list = None, opacities: list = None):
        """
        Plot several frames of the self.u overlapping.
        
        Args:
            - list_indices: a list of integers, each an frame index to be displayed
            - colors: a list of colors (must be same length as list_indices) or None (default)
            - opacities: a list of opacities (must be same length as list_indices) or None (default)
        
        """
        
        for li, list_i in enumerate(list_indices):
            # add optional parameters
            kwargs = self._add_optional_representation_parameters(li, colors, opacities)
                
            self.plot_ith_frame(list_i, **kwargs)

        return self.view




In [96]:
"""
Some important indices you may wanna plot:
- eigenvectors of SQRA
"""

class PostCalculationEvaluator:
    
    """
    Combine the following (read or use saved if you have it available):
        - trajectory or pseudotrajectory (in form of mda Universe)
        - energies
        - full grid (FullGrid object)
        - transition model(MSM or SQRA object)
        
    
    """
    
    def __init__(self, name_pt, default_atom_selection):
        self.name_pt = name_pt
        self.default_atom_selection = default_atom_selection
        self.u = self.read_u_PT()
        self.parsed_trajectory = self.read_parsed_trajectory_PT()
        self.fg = self.read_fg_PT()
        self.energy = self.read_energy_PT()
        self.transition_model = self.read_sqra_PT()
        
        # just to check if everything is sane
        self._assert_consistent_len()
    
    def _assert_consistent_len(self):
        traj_len = len(self.u.trajectory)
        fg_len = len(self.fg.get_full_grid_as_array())
        energy_len = len(self.energy.energies)
        sqra_len = len(self.transition_model.get_transitions_matrix()[0])
        assert traj_len == fg_len == energy_len == sqra_len
        
    """
    --------------------------------------------------------------------------------------------------
                               Readers to set up all necessary structures.
    --------------------------------------------------------------------------------------------------
    """
    
    def __len__(self):
        traj_len = len(self.u.trajectory)
        return traj_len
    
    def read_u_PT(self):
        return mda.Universe(f"{PATH_OUTPUT_PT}{self.name_pt}.gro", f"{PATH_OUTPUT_PT}{self.name_pt}.xtc")

    def read_parsed_trajectory_PT(self):
        pt_parser = FileParser(f"{PATH_OUTPUT_PT}{self.name_pt}.gro", f"{PATH_OUTPUT_PT}{self.name_pt}.xtc")
        pt = pt_parser.get_parsed_trajectory(default_atom_selection=self.default_atom_selection)
        pt.energies = self.read_energy_PT()
        return pt

    def read_fg_PT(self):

        input_names = None
        full_grid_name = None

        # first step: read the name of the full grid from the log file
        with open(f"{PATH_OUTPUT_LOGGING}{self.name_pt}.log") as f:
            while input_names is None or full_grid_name is None:
                line = f.readline()
                if line.startswith("INFO:PtLogger:input grid parameters:"):
                    input_names = line.strip().split(": ")[-1]
                elif line.startswith("INFO:PtLogger:full grid name:"):
                    full_grid_name = line.strip().split(": ")[-1]
                    
        self.grid_name = full_grid_name
        
        input_names = input_names.split(" ")
        t_input = " ".join(input_names[2:])
        fg = FullGrid(o_grid_name=input_names[0], b_grid_name=input_names[1], t_grid_name=t_input,
                      use_saved=True)

        # second step: load the .npy file with the found name
        used_grid = np.load(f"{OUTPUT_PLOTTING_DATA}get_full_grid_as_array_{full_grid_name}.npy")

        # third step: assert that this is actually the grid that has been used
        assert np.allclose(used_grid, fg.get_full_grid_as_array())

        return fg

    def read_energy_PT(self):
        my_parser = XVGParser(f"/home/hanaz63/nobackup/gromacs/{self.name_pt}/{self.name_pt}.xvg")
        return my_parser.get_parsed_energy()

    def read_sqra_PT(self):
        sh = SimulationHistogram(self.parsed_trajectory, self.fg)
        return SQRA(sh, use_saved=True)
    
    """
    --------------------------------------------------------------------------------------------------
                               Getters to obtain important indices.
    --------------------------------------------------------------------------------------------------
    """    
    
    def get_indices_k_lowest_energies(self, k: int, energy_type: str):
        all_energies = self.energy.get_energies(energy_type)
        return k_argmin_in_array(all_energies, k)
    
    def get_indices_neighbours_of_cell_i(self, i: int):
        adj_array = csr_array(self.fg.get_full_adjacency())[:, [i]].toarray().T[0]
        neighbour_indices = np.nonzero(adj_array)[0]
        return neighbour_indices
    
    def get_indices_same_orientation(self, quaternion_grid_index: int):
        num_positions = self.fg.o_rotations.get_N() * self.fg.t_grid.get_N_trans()
        num_quaternions = self.fg.b_rotations.get_N()
        
        return list(range(quaternion_grid_index, len(self), num_quaternions))
    
    def get_indices_same_position(self, position_grid_index: int):
        num_quaternions = self.fg.b_rotations.get_N()

        return list(range(position_grid_index*num_quaternions, (position_grid_index+1)*num_quaternions))

    """
    --------------------------------------------------------------------------------------------------
                               Getters to obtain a measure of magnitude.
    --------------------------------------------------------------------------------------------------
    """
    
    def get_magnitude_energy(self, energy_type: str):
        return self.energy.get_energies(energy_type)
    
    def get_magnitude_ith_eigenvector(self, i: int):
        evalu, evec = self.transition_model.get_eigenval_eigenvec()
        my_eigenvector = evec[0].T[i]
        return my_eigenvector



class MplColorHelper:

    def __init__(self):
        self.cmap = matplotlib.cm.get_cmap('bwr')
        self.norm = matplotlib.colors.TwoSlopeNorm(vcenter=0)
        self.scalarMap = matplotlib.cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

    def get_hex(self, val):
        rgba = self.scalarMap.to_rgba(val)
        return matplotlib.colors.rgb2hex(rgba)
        

In [ ]:
# EXAMPLE - 

# changeable parameters
my_name1 = "H2O"
my_name2 = "H2O"
my_num = "0099"

my_name = f"{my_name1}_{my_name2}_{my_num}"
my_selection = "bynum 4:6"


# read everything from files 
pce = PostCalculationEvaluator(my_name, my_selection)


# display
vm = ViewManager(pce.u)
vm.fresh_view()

# magnitudes
mch = MplColorHelper()
magnitudes = pce.get_magnitude_energy("Potential")


same_ori = pce.get_indices_same_orientation(2)
colors = np.array([mch.get_hex(mag) for mag in magnitudes[same_ori]])
vm.plot_frames_overlapping(same_ori, colors=colors, opacities=[0.5]*len(same_ori))

In [ ]:
# EXAMPLE - eigenvector

# display
vm = ViewManager(pce.u)
vm.fresh_view()

# magnitudes
mch = MplColorHelper()
magnitudes = pce.get_magnitude_ith_eigenvector(0)

same_ori = pce.get_indices_same_position(20)
colors = np.array([mch.get_hex(mag) for mag in magnitudes[same_ori]])
vm.plot_frames_overlapping(same_ori, colors=colors, opacities=[0.5]*len(same_ori))